# Scrapper para obtener información de búsqueda de google (LinkdIn y/o Facebook)

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
import pandas as pd

## Conexión al navegador

In [187]:
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)
start_url = 'https://google.com'
driver.get(start_url)
# Modificar el tamaño de la ventana
#driver.execute_script("document.body.style.zoom='50%'")

## Lectura de archivos csv para obtener nombres

In [171]:
df_3 = pd.read_csv('ejemplo.csv', keep_default_na=False) 
df_3['NombreCompleto'] = df_3.apply(lambda row: f"{row['Nombre']} {row['Paterno']} {row['Materno']}", axis=1)
df_3

,Nombre,Paterno,Materno,NombreCompleto
0,Bryan Antonio,Polito,Palma,Bryan Antonio Polito Palma
1,Luis,Cruz,Jaramillo,Luis Cruz Jaramillo
2,Bryan Antonio,Polito,Palma,Bryan Antonio Polito Palma
3,José Alberto,Montán,López,José Alberto Montán López
4,Andres Manuel,Lopez,Obrador,Andres Manuel Lopez Obrador
5,Jorge Luis,Lopez,,Jorge Luis Lopez
6,Juan de Dios,Olivares,Jimenez,Juan de Dios Olivares Jimenez
7,Bryan Antonio,Polito,Palma,Bryan Antonio Polito Palma
8,José Alberto,Montán,López,José Alberto Montán López


## Scrapper

In [194]:
data_ = []
data_2 = []
for i in df_3['NombreCompleto']:
    nombre = driver.find_element(By.XPATH, '//*[@id="APjFqb"]')
    nombre.send_keys(i)
    nombre.send_keys(Keys.ENTER)
        
    time.sleep(5)
    html_link = driver.page_source
    soup = BeautifulSoup(html_link, 'html.parser')

    href_tags = soup.find_all('a', href= True)
    div_tags = soup.find_all('div', class_="VwiC3b")

    for tag in href_tags:
        href = tag['href']
        text = tag.text

        data_.append({'href':href, 'Text': text})

    for tag in div_tags:
        #div = tag['div']
        resumen = tag.text

        data_2.append({'Resumen': resumen})
        
    google = driver.find_element(By.XPATH, '//*[@id="logo"]')
    google.click()

df= pd.DataFrame(data_)
df_2= pd.DataFrame(data_2)

    #with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        #print(df)

indice = df[df['Text'] == 'Buscar en Ayuda'].index[0]
indices_eliminar = df[df['Text'].isin(['Ayuda', 'Privacidad', 'Condiciones'])].index
df['Text'] = df['Text'].str.strip()
df= df.loc[indice+1:, :].loc[df['href'].str.startswith('http')]
df = df[~df['href'].str.contains('translate')]
df= df.drop(indices_eliminar)


df.reset_index(inplace=True, drop=True)
df = pd.concat([df,df_2], axis=1)
df = df.dropna()
df = df[df['href'].str.contains('facebook|twitter|instagram|linkedin', case=False)]

print(df)

df.to_csv('dataframe.csv', index = False)

                                                 href  \
0   https://mx.linkedin.com/in/polito-palma-bryan-...   
1    https://mx.linkedin.com/in/bryan-palma-296037272   
20  https://m.facebook.com/people/Luis-Cruz-Jarami...   
24          https://www.instagram.com/jose_cruz.1320/   
25  https://mx.linkedin.com/in/oscar-enrique-cruz-...   
26  https://es.linkedin.com/in/alejandra-de-la-cru...   
36  https://mx.linkedin.com/in/polito-palma-bryan-...   
37   https://mx.linkedin.com/in/bryan-palma-296037272   
54  https://mx.linkedin.com/in/jos%C3%A9-alberto-m...   
57  https://es-es.facebook.com/JoaquinLopezDoriga/...   

                                                 Text  \
0   Bryan Antonio Polito Palma | LinkedInlinkedin....   
1   Bryan Palma - Universidad Tecnológica de Méxic...   
20  Luis Cruz Jaramillo | Facebookfacebook.comhttp...   
24  Jose Luis Cruz Jaramillo (@jose_cruz.1320) • I...   
25  Oscar Enrique Cruz Jaramillo - Material Launch...   
26  Alejandra de la Cruz Jaram

In [232]:
data = []
data_ = []
html_link = driver.page_source
soup = BeautifulSoup(html_link, 'html.parser')
div = soup.find_all('div', class_="kvH3mc BToiNc UK95Uc")
cont = soup.find_all('div', class_ = "Z26q7c UK95Uc")

for tag in div:
    href_tags = tag.find_all('a', href=True)
    text = tag.text
    
    for href_tag in href_tags:
        href = href_tag['href']
        
        data.append({'href': href, 'Text': text})

for tag in cont:
    span_tags = tag.find_all('span')
    
    if 'ITHCWe' in span_tag.get('class', []):
            continue
    
    if len (span_tags) > 1:
        text = ' '.join(span_tag.text for span_tag in span_tags)
    
    elif len(span_tags) == 1:
        text = span_tags[0].text
    else:
        text = tag.text.strip()
        
        
data_.append({'Text': text})
    


df = pd.DataFrame(data)
df_2 = pd.DataFrame(data_)

print(df)
print(df_2)

                                                href  \
0                       https://lopezobrador.org.mx/   
1  https://www.gob.mx/presidencia/estructuras/and...   
2  https://www.youtube.com/channel/UCxEgOKuI-n-WO...   
3  https://es.wikipedia.org/wiki/Andr%C3%A9s_Manu...   
4    https://es-la.facebook.com/lopezobrador.org.mx/   
5   https://www.instagram.com/lopezobrador/?hl=es-la   
6  https://elpais.com/noticias/andres-manuel-lope...   
7      https://www.bbc.com/mundo/topics/cyx5kr6gdy1t   
8  https://www.eleconomista.com.mx/personaje/Andr...   

                                                Text  
0  AMLO – Sitio Oficial de Andrés Manuel López Ob...  
1  Andrés Manuel López Obrador | Presidencia de l...  
2  Andrés Manuel López Obrador - YouTubeyoutube.c...  
3  Andrés Manuel López Obrador - Wikipedia, la en...  
4  Andrés Manuel López Obrador - Facebookfacebook...  
5  Andrés Manuel López Obrador (@lopezobrador ......  
6  Andrés Manuel López Obrador en EL PAÍS - AMLOe...  

In [233]:
data = []
data_ = []
html_link = driver.page_source
soup = BeautifulSoup(html_link, 'html.parser')
href = soup.find_all('a', href= True)
div = soup.find_all('div', class_="VwiC3b")
class_ = soup.find_all('div', class_="kvH3mc BToiNc UK95Uc")

for tag in class_:
    href = tag.find_all('a', href = True)
    text = tag.text

    data.append({'href':href, 'Text': text})

for tag in div:
    #div = tag['div']
    text = tag.text

    data_.append({'Text': text})

df= pd.DataFrame(data)
df.drop
df_2= pd.DataFrame(data_)

#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #print(df)

indice = df[df['Text'] == 'Buscar en Ayuda'].index[0]
indices_eliminar = df[df['Text'].isin(['Ayuda', 'Privacidad', 'Condiciones'])].index
df['Text'] = df['Text'].str.strip()
df= df.loc[indice+1:, :].loc[df['href'].str.startswith('http')]
df = df[~df['href'].str.contains('translate')]
df= df.drop(indices_eliminar)

df.reset_index(inplace=True, drop=True)
df = pd.concat([df,df_2.rename(columns={'Text':'Text2'})], axis=1)
#df = df[df['href'].str.contains('facebook|twitter|instagram|linkedin', case=False)]
print(df_2)
df.to_csv('dataframe.csv')

IndexError: index 0 is out of bounds for axis 0 with size 0

In [186]:
data = []
data_2 = []
for i in df_3['NombreCompleto']:
    nombre = driver.find_element(By.XPATH, '//*[@id="APjFqb"]')
    nombre.send_keys(i)
    nombre.send_keys(Keys.ENTER)
        
    time.sleep(5)
    html_link = driver.page_source
    soup = BeautifulSoup(html_link, 'html.parser')

    href_tags = soup.find_all('a', href=True)
    div_tags = soup.find_all('div', class_="VwiC3b")
    class_tags = soup.find_all('div', class_="MjjYud")

    for tag in href_tags:
        href = tag['href']
        text = tag.text
        data.append({'href': href, 'Text': text})

    for tag in div_tags:
        text_2 = tag.text
        data_2.append({'Text2': text_2})

    google = driver.find_element(By.XPATH, '//*[@id="tsf"]/div[1]/div[1]/div[2]')
    google.click()

df = pd.DataFrame(data)
df_2 = pd.DataFrame(data_2)

indice = df[df['Text'] == 'Buscar en Ayuda'].index[0]
indices_eliminar = df[df['Text'].isin(['Ayuda', 'Privacidad', 'Condiciones'])].index
df['Text'] = df['Text'].str.strip()
df = df.loc[indice+1:, :]
df = df[df['href'].str.startswith('http')]
df = df.drop(indices_eliminar)

df.reset_index(inplace=True, drop=True)
df = pd.concat([df, df_2], axis=1)
#df = df[df['href'].str.contains('facebook|twitter|instagram|linkedin', case=False)]
df = df.dropna()

print(df)

df.to_csv('dataframe.csv', index = False)


KeyboardInterrupt: 